# 05. LoRAによるファインチューニング (学習と永続化)

このノートブックでは、LoRA（QLoRA）による学習を行い、その成果物を **Google Driveに保存** します。これにより、別の日に行われる統合演習（07）で、学習したモデルを確実に再利用できます。

## 事前準備
Google Colabのメニュー「ランタイム」→「ランタイムのタイプを変更」で **T4 GPU** を選択してください。

In [ ]:
# 編集禁止セル
import os
import sys
import torch
from google.colab import drive

# 1. Google Driveのマウント (永続保存に必須)
if not os.path.isdir('/content/drive'): drive.mount('/content/drive')

repo_path = '/content/llm_lab'
if os.path.exists(repo_path): !rm -rf {repo_path}
!git clone -b stable-base https://github.com/akio-kobayashi/llm_lab.git {repo_path}
os.chdir(repo_path)

!pip install -q -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu peft trl datasets gradio
if 'src' not in sys.path: sys.path.append(os.path.abspath('src'))

from src.common import load_llm, generate_text
from src.lora import create_lora_model, train_lora
from peft import PeftModel
print('セットアップが完了しました。')

## 1. ベースモデルのロードと学習前の確認

In [ ]:
# 編集禁止セル
base_model, tokenizer = load_llm(use_4bit=True)

question = "『星屑のメモリー』の主人公について教えて。"
instruction = "回答は必ず以下のJSON形式で出力してください。\n{ \"answer\": \"...\", \"confidence\": \"high|medium|low\" }"
prompt = f"以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n{question}\n{instruction}\n\n### 応答:\n"

print("--- 学習前の回答 ---")
res = generate_text(base_model, tokenizer, prompt, max_new_tokens=128)
print(res.split("### 応答:")[-1].strip())

## 2. LoRA学習の実行 (成果をGoogle Driveへ保存)

学習したアダプタを Google Drive の `llm_lab_outputs/demo_adapter` に保存します。

In [ ]:
# 編集禁止セル
# 保存先パスを Google Drive 上に完全固定
OUTPUT_BASE_DIR = "/content/drive/MyDrive/llm_lab_outputs/demo_adapter"
os.makedirs(OUTPUT_BASE_DIR, exist_ok=True)

print("1. LoRAアダプタをモデルに追加中...")
lora_model = create_lora_model(base_model)

print("2. 簡易学習を開始します（約3〜5分）...")
train_lora(
    model=lora_model,
    tokenizer=tokenizer,
    train_dataset_path='data/lora/lora_train_sample.jsonl',
    output_dir=OUTPUT_BASE_DIR,
    max_steps=40,
    learning_rate=5e-5
)

print(f"学習完了。成果物は Google Drive に永続保存されました: {OUTPUT_BASE_DIR}")

## 3. 保存されたアダプタのロードテスト

セッションが切れた後の状態を想定し、**Google Drive から直接** アダプタをロードして動作を確認します。

In [ ]:
# 編集禁止セル
FINAL_ADAPTER_PATH = os.path.join(OUTPUT_BASE_DIR, "final_adapter")

print(f"Google Driveからアダプタをロードしています: {FINAL_ADAPTER_PATH}")
test_model = PeftModel.from_pretrained(base_model, FINAL_ADAPTER_PATH)
test_model.eval()

print("--- 学習後の回答 (Google Driveからロード) ---")
res = generate_text(test_model, tokenizer, prompt, max_new_tokens=128)
print(res.split("### 応答:")[-1].strip())

## まとめ
Google Driveに保存されたアダプタは、次回の **07. 統合デモ** でそのまま読み込んで使用できます。